In [77]:
import requests
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
import time

In [2]:
r = requests.get('https://www.hse.ru/org/persons/?udept=135213')
page = BeautifulSoup(r.text, 'html.parser')
url = 'https://www.hse.ru'
letters = []
for link in page.findAll("div", {'class': "abc-filter__letter"}):
    if (str(link.contents)).find('"') != -1:
            s = url + '/org/persons/' + str(link.contents)[10:34]
            letters.append(str(s))
print(len(letters))

25


In [4]:
people=[]
for letter in tqdm(letters):
    r = requests.get(letter)
    page = BeautifulSoup(r.text, 'html.parser')
    url = 'https://www.hse.ru'
    for link in page.findAll("a", {'class':"link"}):
        try:
            if (link['href'][:13]=="/org/persons/" or link['href'][:7]=="/staff/") and link['href'][13]!='?' and link.text != 'полный список':
                people.append({
                    'name': link.text[1:],
                    'link': url + link['href']
                })
        except:
            a=1
df = pd.DataFrame(people)
df.head(10)

100%|██████████| 25/25 [01:07<00:00,  2.71s/it]


,name,link
0,Абашева Александра Сергеевна,https://www.hse.ru/org/persons/485801
1,Авраменко Иван Александрович,https://www.hse.ru/org/persons/99247481
2,Агаркова Наталия Владиславовна,https://www.hse.ru/org/persons/28123133
3,Айхбергер Юрген Томас Германн,https://www.hse.ru/org/persons/359359486
4,Александров Владимир Анатольевич,https://www.hse.ru/org/persons/420359428
5,Алексеева Лариса Николаевна,https://www.hse.ru/org/persons/66770114
6,Аленина Карина Анатольевна,https://www.hse.ru/org/persons/202287
7,Алова Надежда Владимировна,https://www.hse.ru/org/persons/28123313
8,Андреева Ольга Юрьевна,https://www.hse.ru/org/persons/152893083
9,Андрианов Игорь Владимирович,https://www.hse.ru/org/persons/223725475


In [91]:
vkr = []
for person in tqdm(people):
    r = requests.get(person['link'])
    page = BeautifulSoup(r.text, 'html.parser')
    for link in page.findAll("a", {'class':"link"}):
            if link['href'][:9]=='/edu/vkr/':
                    #print(url+link['href'])
                    vkr.append(url+link['href'])

100%|██████████| 321/321 [09:59<00:00,  1.87s/it]


In [92]:
browser = webdriver.Firefox(executable_path = './.browserDrivers/geckodriver')
vkr_data = []
for link in tqdm(vkr):
    #print(link)
    browser.get(link)
    browser.implicitly_wait(1)
    time.sleep(1)
    page = BeautifulSoup(browser.page_source, 'html.parser')
    vkr_tmp = []
    name_vkr=''
    name_student=''
    name_prof=''
    campus=''
    program=''
    grade=-1
    year=-1
    link_file=''
    for elem in page.findAll("h1"):
        #print(elem.text)
        name=elem.text
    for elem in page.findAll("p",{'class':'vkr-card__item'}):
        #print(elem.text)
        elem_name = elem.text.split(':')[0]
        elem_text = elem.text.split(':')[1]
        if elem_name ==' ФИО студента':
            name_student = elem_text
        elif elem_name ==' Руководитель':
            name_prof = elem_text
        elif elem_name ==' Кампус / факультет':
            campus = elem_text
        elif elem_name ==' Программа':
            program = elem_text
        elif elem_name ==' Год защиты':
            year = elem_text
        elif elem_name ==' Оценка':
            grade = elem_text
            
    for elem in page.findAll("a", {'class':'link'}):
        if elem['href'][:17]=='http://lms.hse.ru':
            #print(elem['href'])
            link_file=elem['href']
    
    vkr_data.append({
        'link_vkr':link,
        'name_vkr':name_vkr,
        'name_student':name_student,
        'name_prof':name_prof,
        'campus':campus,
        'programm':program,
        'grade':grade,
        'year':year,
        'link_file':link_file
    })

100%|██████████| 1708/1708 [45:26<00:00,  1.60s/it] 


In [28]:
url = 'http://lms.hse.ru/ap_service.php?getwork=1&guid=D6A3C344-D35B-4820-8514-EF33F306E5E3'
r = requests.get(url, allow_redirects=True)

open('test.docx', 'wb').write(r.content)

916468

In [93]:
tmp = pd.DataFrame(vkr_data)
tmp.head(11)

,link_vkr,name_vkr,name_student,name_prof,campus,programm,grade,year,link_file
0,https://www.hse.ru/edu/vkr/219524501,,Дроздова Ирина Валерьевна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),-1,2018,
1,https://www.hse.ru/edu/vkr/219524122,,Стерледева Наталья Владимировна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),7,2018,http://lms.hse.ru/ap_service.php?getwork=1&gui...
2,https://www.hse.ru/edu/vkr/153009903,,Беленова Анастасия Александровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,
3,https://www.hse.ru/edu/vkr/153009689,,Воронков Кирилл Евгеньевич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,
4,https://www.hse.ru/edu/vkr/153009541,,Бадягина Мария Германовна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,
5,https://www.hse.ru/edu/vkr/153009117,,Шевелева Анастасия Андреевна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,
6,https://www.hse.ru/edu/vkr/153008778,,Грибанов Максим Игоревич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,
7,https://www.hse.ru/edu/vkr/153008524,,Щекотова Алиса Сергеевна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,
8,https://www.hse.ru/edu/vkr/153006804,,Метелева Юлия Илдаровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,
9,https://www.hse.ru/edu/vkr/123819138,,Миненко Анна Владимировна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Бакалавриат,-1,2014,


In [94]:
import os
if not os.path.exists('./data'):
    os.makedirs('./data')
tmp.to_csv('./data/vkr.csv')
df.to_csv('./data/people.csv')

In [96]:
tmp['link_file'].nunique()

501